# Model Saving

### Model Name Config

In [1]:
MODEL_NAME = 'xlm-roberta-base'
saved_model_name = "xlm-r_88acc_best_model_v2"

In [9]:
MODEL_NAME = 'bert-base-multilingual-cased'
saved_model_name = "mbert_80pacc_try_model_v1"

In [4]:
MODEL_NAME = 'bert-base-uncased'
saved_model_name = "bert-base_best_model_v1"
load_checkpoint_name = "bert-base__lr3e-05_ep100_bs64.h5"

### Load Model

In [5]:
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2
)

model.load_weights('checkpoints/' + load_checkpoint_name)


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Save Model Type

In [ ]:
# Hugging Face Transformer Format
model.save_pretrained("saved_model/"+saved_model_name, save_format="tf")

In [ ]:
# # Keras Format
model.save("saved_model/"+saved_model_name)

In [6]:
# Standard Tensorflow Format
import tensorflow as tf
tf.saved_model.save(model, "saved_model/"+saved_model_name)

INFO:tensorflow:Assets written to: saved_model/bert-base_best_model_v1/assets


INFO:tensorflow:Assets written to: saved_model/bert-base_best_model_v1/assets


# Tokenizer Saving

In [7]:
tokenizer_name = "xlm-roberta-base"
tokenizer_saved_name = "for_" + "xmlr-pd-best_fix_v1"

In [14]:
tokenizer_name = "bert-base-multilingual-cased"
tokenizer_saved_name = "for_" + "mbert_80pacc_try_model_v1"

In [16]:
from transformers import AutoTokenizer

# Assume you already have a tokenizer (for instance, fine-tuned or pretrained)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

# Save the tokenizer to a local directory
tokenizer.save_pretrained("saved_tokenizer/" + tokenizer_saved_name)

('saved_tokenizer/for_mbert_80pacc_try_model_v1/tokenizer_config.json',
 'saved_tokenizer/for_mbert_80pacc_try_model_v1/special_tokens_map.json',
 'saved_tokenizer/for_mbert_80pacc_try_model_v1/vocab.txt',
 'saved_tokenizer/for_mbert_80pacc_try_model_v1/added_tokens.json',
 'saved_tokenizer/for_mbert_80pacc_try_model_v1/tokenizer.json')